In [1]:
import numpy as np
import pandas as pd
from keras.models import *
from keras.layers import *
from tensorflow.keras.utils import to_categorical
from sklearn.utils import shuffle

In [2]:
dataset=pd.read_csv('dataset.csv')

In [3]:
dataset

,sentence,label
0,I love this movie!,2
1,This book is so boring.,0
2,The weather today is okay.,1
3,I had a terrible experience at the restaurant.,0
4,The new album is fantastic!,2
...,...,...
2293,The customer support at this company is satisf...,1
2294,The food at this restaurant is quite tasty.,2
2295,I'm feeling somewhat indifferent about the new...,1
2296,I'm feeling really positive about the progress...,2


In [4]:
duplicate = dataset[dataset.duplicated()]

In [5]:
duplicate

,sentence,label
104,I'm feeling indifferent about the outcome.,1
115,I'm neutral about the situation.,1
155,The traffic was unbearable.,0
156,I'm ecstatic about the news.,2
157,The concert was amazing!,2
...,...,...
2293,The customer support at this company is satisf...,1
2294,The food at this restaurant is quite tasty.,2
2295,I'm feeling somewhat indifferent about the new...,1
2296,I'm feeling really positive about the progress...,2


In [6]:
dataset.drop_duplicates(keep=False,inplace=True)

In [7]:
dataset

,sentence,label
0,I love this movie!,2
1,This book is so boring.,0
2,The weather today is okay.,1
3,I had a terrible experience at the restaurant.,0
4,The new album is fantastic!,2
...,...,...
2223,I'm really looking forward to the upcoming hol...,2
2225,The food at this restaurant is decent.,1
2226,I'm feeling pretty optimistic about the future.,2
2227,The service at this establishment is subpar.,0


In [8]:
dataset=shuffle(dataset)

In [9]:
dataset

,sentence,label
1643,"That new restaurant downtown is amazing, I lov...",2
1772,The long wait to get seated at the restaurant ...,0
1954,Going for a walk in the forest was so calming.,2
55,The vacation was relaxing.,2
1686,"I'm indifferent to the new office furniture, i...",1
...,...,...
1826,The long lines at the grocery store were reall...,0
1679,Going to the beach and feeling the sand betwee...,2
2108,"I'm neutral about the new menu at the cafeteria.""",1
818,I'm ecstatic about the news of her promotion.,2


In [10]:
X_data=dataset['sentence'].values
labels=dataset['label'].values

In [11]:
X_data.shape,labels.shape

((1061,), (1061,))

In [12]:
split=0.2
X_train=X_data[(int)(split*X_data.shape[0]):]
X_test=X_data[:(int)(split*X_data.shape[0])]
Y_train=labels[int(split*labels.shape[0]):]
Y_test=labels[:int(split*labels.shape[0])]

In [13]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(849,) (212,) (849,) (212,)


In [14]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1061 entries, 1643 to 126
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   sentence  1061 non-null   object
 1   label     1061 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 24.9+ KB


In [15]:
np.unique(labels)

array([0, 1, 2], dtype=int64)

In [16]:
Y_test=to_categorical(Y_test)
Y_train=to_categorical(Y_train)

In [17]:
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

(849,) (212,) (849, 3) (212, 3)


In [18]:
f=open('D:/Skills/Emoji Classifier/glove.6B.50d.txt',encoding='utf-8')
embedding_index={}

for line in f:
    values=line.split()
    word=values[0]
    coefs=np.asarray(values[1:],dtype='float')
    embedding_index[word]=coefs
    
f.close()

In [19]:
def embedding_output(X):
    maxLen=50 #max words in sentence
    embedding_out=np.zeros((X.shape[0],maxLen,50))
    for ix in range(X.shape[0]):
        # print(ix)
        try:
            X[ix]=X[ix].split()
        except:
            pass
        for ij in range(maxLen):
            try:
                embedding_out[ix][ij]=embedding_index[X[ix][ij].lower()]
            except:
                embedding_out[ix][ij]=np.zeros((50,))
    return embedding_out

In [20]:
X_train=embedding_output(X_train)
X_test=embedding_output(X_test)

In [21]:
X_train.shape,Y_train.shape,X_test.shape,Y_test.shape

((849, 50, 50), (849, 3), (212, 50, 50), (212, 3))

In [22]:
model=Sequential()
model.add(LSTM(50,input_shape=(50,50),return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(20,return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(10,return_sequences=False))
model.add(Dropout(0.1))
model.add(Dense(3,activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 50, 50)            20200     
                                                                 
 dropout (Dropout)           (None, 50, 50)            0         
                                                                 
 lstm_1 (LSTM)               (None, 50, 20)            5680      
                                                                 
 dropout_1 (Dropout)         (None, 50, 20)            0         
                                                                 
 lstm_2 (LSTM)               (None, 10)                1240      
                                                                 
 dropout_2 (Dropout)         (None, 10)                0         
                                                                 
 dense (Dense)               (None, 3)                 3

In [24]:
hist=model.fit(X_train,Y_train,epochs=25,batch_size=50,shuffle=True,validation_split=0.2)

Epoch 1/25
14/14 [==============================] - 6s 107ms/step - loss: 1.0725 - acc: 0.4418 - val_loss: 1.0503 - val_acc: 0.4765
Epoch 2/25
14/14 [==============================] - 0s 33ms/step - loss: 1.0272 - acc: 0.4448 - val_loss: 0.8948 - val_acc: 0.6353
Epoch 3/25
14/14 [==============================] - 0s 35ms/step - loss: 0.8549 - acc: 0.5876 - val_loss: 0.9425 - val_acc: 0.5000
Epoch 4/25
14/14 [==============================] - 0s 33ms/step - loss: 0.8429 - acc: 0.5714 - val_loss: 0.7605 - val_acc: 0.6412
Epoch 5/25
14/14 [==============================] - 1s 37ms/step - loss: 0.7593 - acc: 0.6127 - val_loss: 0.7413 - val_acc: 0.6412
Epoch 6/25
14/14 [==============================] - 1s 42ms/step - loss: 0.7305 - acc: 0.6289 - val_loss: 0.6964 - val_acc: 0.6412
Epoch 7/25
14/14 [==============================] - 1s 52ms/step - loss: 0.6873 - acc: 0.6436 - val_loss: 0.7060 - val_acc: 0.6412
Epoch 8/25
14/14 [==============================] - 0s 34ms/step - loss: 0.6691 - 

In [25]:
model.evaluate(X_test,Y_test)

7/7 [==============================] - 0s 10ms/step - loss: 0.4118 - acc: 0.8632


[0.41180214285850525, 0.8632075190544128]

In [32]:
# model.save('model.h5')

D:\anaconda\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
def predict(X):
    X=np.asarray([X])
    print(X.shape)
    X=embedding_output(X)
    print(model.predict(X))
    return np.argmax(model.predict(X))

In [28]:
sent='He is a good student'
sent1='He is the bad student'
sent2='he is ok student'

In [29]:
print(predict(sent))

(1,)
1/1 [==============================] - 1s 873ms/step
[[0.20561226 0.01474614 0.77964157]]
1/1 [==============================] - 0s 8ms/step
2


In [30]:
print(predict(sent1))

(1,)
1/1 [==============================] - 0s 24ms/step
[[0.56523114 0.36836687 0.066402  ]]
1/1 [==============================] - 0s 17ms/step
0


In [31]:
print(predict(sent2))

(1,)
1/1 [==============================] - 0s 23ms/step
[[0.80896175 0.07095587 0.12008242]]
1/1 [==============================] - 0s 21ms/step
0


In [2]:
model=load_model('model.h5')

In [3]:
import joblib
joblib.dump(model,'sentiment_model.pkl')

['sentiment_model.pkl']